# 0. Configs and Imports

## 0.1. Imports

In [1]:
# Import for directory creation
from src.utils.file_operators import create_directory_structure

# Imports for data fetch
from src.data_collectors.yahoo_fin_collector import fetch_data

# Imports for loading config
from src.utils.file_operators import load_yaml

# Imports for Correlation Calculation
from src.computation.correlation import calculate_pairwise_corrrelation

# Imports for Portfolio Unierse Generation
from src.computation.ticker_universe_generator import create_portfolio_universe

## 0.2. Base file path, Configuration Filepath & Configuration Dictionary

In [2]:
base_file_path = "C:/Users/sharv/Documents/Sharvil/Projects/AutoWeight/data"

In [3]:
conf_path = "C:/Users/sharv/Documents/Sharvil/Projects/AutoWeight/config-sd.yaml"

In [4]:
config = load_yaml(conf_path)

## 0.3. Create Directory Structure (Optional: Run once only)

In [5]:
create_directory_structure(path=base_file_path, verbose=True)

INFO: Creating Base Directory
INFO: Base Directory path exists: C:\Users\sharv\Documents\Sharvil\Projects\AutoWeight\data
INFO: Creating Sub Directories
INFO: Sub Directory path exists: C:\Users\sharv\Documents\Sharvil\Projects\AutoWeight\data\01_raw

INFO: Sub Directory path exists: C:\Users\sharv\Documents\Sharvil\Projects\AutoWeight\data\02_processed

INFO: Sub Directory path exists: C:\Users\sharv\Documents\Sharvil\Projects\AutoWeight\data\03_analysis

SUCCESS: Created all directories successfully!



# 1. Data Fetch

## 1.1. Validate the configurations for data fecth

In [6]:
# Print the tickers defined in the config
ticker_list = config['config']['data']['tickers']

# Print the window and volitility_indicator defined in the config
window = config['config']['data']['window_in_days']
volatility_indicator = config['config']['data']['vix']

In [7]:
print("The tickers in the config list are:")

for idx, ticker in enumerate(ticker_list):
    print(f"    {idx+1}. {ticker}")

print(f"The window of data to evaluate is `{window}` days")

print(f"The volatility indicator used is `{volatility_indicator}`")

The tickers in the config list are:
    1. HDFCSML250.NS
    2. HDFCGOLD.NS
    3. HDFCNEXT50.NS
    4. AMBUJACEM.NS
    5. ASIANPAINT.NS
    6. EIDPARRY.NS
    7. ETERNAL.NS
    8. GMRAIRPORT.NS
    9. HDFCBANK.NS
    10. ABC.NS
    11. ITC.NS
    12. LLOYDSME.NS
    13. NBCC.NS
    14. OLECTRA.NS
    15. PCBL.NS
The window of data to evaluate is `900` days
The volatility indicator used is `^INDIAVIX`


## 1.2. Run the `fetch_data` pipeline

In [8]:
ohlcv_data = fetch_data(
    config_path=conf_path,
    save_data=True, 
    verbose=False
)

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ABC.NS"}}}
$ABC.NS: possibly delisted; no timezone found

1 Failed download:
['ABC.NS']: possibly delisted; no timezone found


ERROR: Possible Delising or Incorrect Name for ticker: ABC.NS

SUCCESS: fetch_data pipeline run completed


# 2. Calculate Correlation

## 2.1. Validate the correaltion parameters

In [9]:
correlation_params = config['config']['parameters']['correlation']

In [10]:
print(f"The method used for calculating correlation is: {correlation_params['method']}")
print(f"Minimum samples needed for correlation are: {correlation_params['min_observations']}")
print(f"The price metrics defined inn the configs are:")
for idx, metric in enumerate(correlation_params['metrics']):
    print(f"    {idx+1}. {metric}")


The method used for calculating correlation is: spearman
Minimum samples needed for correlation are: 60
The price metrics defined inn the configs are:
    1. open
    2. high
    3. low
    4. close
    5. adj_close


## 2.2. Run the `calculate_pairwise_corrrelation` pipeline

In [11]:
correlation_results = correlation_results = calculate_pairwise_corrrelation(
    config_path=conf_path,
    save_data=True,
    verbose=True
)

INFO:    Starting Correlation Pipeline for open metric of data
INFO:    Calculating PCT Change for open metric of data
INFO:    Calculating Correlation for PCT Change for open metric of data
INFO:    Adjusting Correlation based on optimization strategy for open metric of data
SUCCESS:    Correlation Pipeline for open metric of dataframes, are saved.
SUCCESS:    Correlation Pipeline for open metric of data finished successfully

INFO:    Starting Correlation Pipeline for high metric of data
INFO:    Calculating PCT Change for high metric of data
INFO:    Calculating Correlation for PCT Change for high metric of data
INFO:    Adjusting Correlation based on optimization strategy for high metric of data
SUCCESS:    Correlation Pipeline for high metric of dataframes, are saved.
SUCCESS:    Correlation Pipeline for high metric of data finished successfully

INFO:    Starting Correlation Pipeline for low metric of data
INFO:    Calculating PCT Change for low metric of data
INFO:    Calculatin

## 2.3. Generate a portfolio Universe using the `create_portfolio_universe` function

In [12]:
anchors = create_portfolio_universe(config_path=conf_path, save_data=True, verbose=True)

INFO: Loading Portfolio Params
SUCCESS: Loaded and Validated Portfolio Params

SUCCESS: Loaded the latest correlation Matrix

INFO: Calculating the Global Correlation for all securities in correlation matrix
INFO: Filtering global correlation to loowest globally correlated 5 anchor points
SUCCESS: Global Correlation for all securities caluclated

INFO: Finding Optimal Buckets


In [13]:
anchors

securities
HDFCGOLD.NS      0.035472
ITC.NS           0.188432
ASIANPAINT.NS    0.190371
ETERNAL.NS       0.236249
HDFCBANK.NS      0.240647
dtype: float64